In [14]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle as pkl
from datetime import datetime
from tqdm import tqdm

#generate random spin configuration. 
def initial(L):
    config = 2*np.random.randint(2, size = (L,L)) - 1 #spin은 +1 or -1
    return config


#Monte carlo algorithm
def mcmh(config, temp):
    L = int(np.sqrt(config.size))
    x = np.random.randint(0,L)
    y = np.random.randint(0,L)
    s = config[x,y]  
    neigh = config[(x+1)%L, y] + config[x,(y+1)%L] + config[(x-1)%L,y] + config[x,(y-1)%L]
    DE = 2 * s * neigh 

    #accept or reject 
    if DE < 0:
        s *= -1 
    elif np.random.rand() < np.exp(- DE*1/temp):
        s *= -1 
    config[x,y] = s
    return config


#Calculate energy of given configuration
def energy(config):
    L = int(np.sqrt(config.size))
    H = 0
    for i in range(len(config)):
        for j in range(len(config)):
            x = np.random.randint(0,L)
            y = np.random.randint(0,L)
            s = config[x,y] 
            neigh = config[(x+1)%L, y] + config[x,(y+1)%L] + config[(x-1)%L,y] + config[x,(y-1)%L]
            H += - neigh * s
    return H/4

#magnetization
def magnetization(config):
    magnet = np.sum(config)
    return magnet

L = 20; #system의 가로or세로 spin의 개수(N = L*L)
equilsteps = 1000; #np.power(2,L);?equilibrium으로 만들기 위한 steps
mcsteps = 1000; #np.power(2,L);? #mcmh를 몇번 iterate할지 결정

dT = 50; #temperature T의 변화 steps
T = np.linspace(2.0, 4.0, dT) #관심있는 온도범위(critical temperature ~ 2.269)
T2 = np.linspace(2.0, 4.0, dT)

n1, n2  = 1.0/(mcsteps*L*L), 1.0/(mcsteps*mcsteps*L*L) 

def get_sample(L, T):
    E, M = 0, 0;
    config = initial(L);
    for i in range(equilsteps): # spin configuration을 equilbrium state로 만듦.
        mcmh(config, T)

    for j in range(mcsteps): #spin을 마구 뒤집어가면서 (average)energy, magnetization등을 구함
        mcmh(config, T)
        Energy = energy(config);
        Mag = magnetization(config);

        E += Energy;
        M += Mag;
    return T, E, M, config


In [ ]:
data3=[]
for l in tqdm(range(10000)):
    data3.append(get_sample(L, 3))
with open('%s_T=3.pkl' %(str(datetime.today())[:10]), 'wb') as f:
    pkl.dump(data3, f)

 26%|██▋       | 2642/10000 [1:39:22<4:31:50,  2.22s/it]

In [ ]:
data5=[]
for l in tqdm(range(10000)):
    data5.append(get_sample(L, 5))
with open('%s_T=5.pkl' %(str(datetime.today())[:10]), 'wb') as f:
    pkl.dump(data5, f)

In [ ]:
data1=[]
for l in tqdm(range(10000)):
    data1.append(get_sample(L, 1))
with open('%s_T=1.pkl' %(str(datetime.today())[:10]), 'wb') as f:
    pkl.dump(data1, f)

In [11]:
get_sample(20, 2)

(2,
 -179219.0,
 2968,
 array([[-1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1,
         -1, -1, -1, -1],
        [ 1,  1,  1,  1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1,
         -1, -1, -1, -1],
        [ 1,  1,  1, -1, -1, -1, -1, -1,  1, -1,  1,  1,  1, -1, -1, -1,
         -1, -1, -1,  1],
        [ 1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1,  1],
        [ 1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,
         -1, -1,  1,  1],
        [ 1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1,
         -1, -1,  1,  1],
        [ 1,  1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1, -1,
         -1, -1,  1,  1],
        [ 1,  1,  1,  1,  1, -1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1],
        [ 1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,
          1,  1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1, 